In [2]:
import datetime
from bwb.db import LocalDB
import bwb.customstrategy as cst
import bwb.indicator as indicator
from backtesting.lib import crossover


## Target issue

In [3]:
issues = ['AAPL']
d = LocalDB()

In [4]:
# # カスタマイズできるインジケータの説明
# print(help(cst.base_params))

# 自分でカスタマイズできるストラテジのクラス
class MyStrategy(cst.CustomStrategy):
    # 今回のストラテジで用いるインジケーターの宣言と設定
    def init(self):
        self.macd, self.macdsignal = self.I(indicator.macd, self.candle, day_short=9, day_long=26, span=9)

    # 売買設定
    def next(self):
        if crossover(self.macd, self.macdsignal):
            self.buy()
        elif crossover(self.macdsignal, self.macd):
            self.position.close()

In [5]:
# いくつも定義できる
class YourStrategy(cst.CustomStrategy):
    # 今回のストラテジで用いるインジケーターの宣言と設定
    def init(self):
        # 利用するMACDの定義
        self.macd, self.macdsignal = self.I(indicator.macd, self.candle, day_short=9, day_long=26, span=9)
        # 利用するボリンジャーバンドの定義
        self.upper, self.lower = self.I(indicator.ci, self.candle, day=20, upper_sigma=1, lower_sigma=1)

    # 売買設定
    def next(self):
        if crossover(self.macd, self.macdsignal):
            self.buy()
        elif crossover(self.macdsignal, self.macd):
            if crossover(self.upper, self.data['Close']):
                self.position.close()

In [6]:
# 銘柄ごとに処理の実行
for issue in issues:
    # 計算する銘柄の表示
    print(issue)
    candle = d.loader(issue, start = '2020/04/01', end = '2021/06/01')
    # Mystrategyの実行と保存
    d.runsaver(MyStrategy, candle, issue)
    # YourStrategyの実行と保存
    d.runsaver(YourStrategy, candle, issue)

AAPL


NameError: name 'crossover' is not defined